In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("car_data_autovit.csv")

In [3]:
df = df.dropna()

In [4]:
df

,Unnamed: 0,brand,model,full_title,engine_displacement_cm3,power_hp,ad_description,price_eur,mileage_km,fuel_type,production_year
0,0,Porsche,Macan,Porsche Macan 3.6 PDK Turbo,3604.0,400.0,"Turbo, Garanție aprobată, Primul proprietar fă...",31499.00,168188.0,Benzina,2014.0
1,1,Mercedes-Benz,GLC Coupe,Mercedes-Benz GLC Coupe 300 e 4Matic 9G-TRONIC...,1991.0,319.0,AMG Trapa Distronic Camere 360 Webasto Burmest...,44757.90,112197.0,Hibrid Plug-In,2020.0
2,2,Renault,Trafic,Renault Trafic 1.6 dCi 115 L1H1 Komfort,1598.0,115.0,1.6 dCi | 2014 | L1H1 | Carte service | ITP No...,7700.00,363404.0,Diesel,2014.0
3,3,Volkswagen,Passat,Volkswagen Passat,1598.0,120.0,1.6 TDI | 07.2018 | Garantie 12 Luni | Distrib...,10990.00,218877.0,Diesel,2018.0
4,4,Ford,Kuga,Ford Kuga 2.0 EcoBlue 4x4 Aut. VIGNALE,1995.0,190.0,Vignale AWD Camere360 ACC Keyless Revizie la z...,25397.90,68781.0,Diesel,2021.0
...,...,...,...,...,...,...,...,...,...,...,...
12695,12695,Skoda,RAPID,Skoda RAPID 1.6 TDI DSG Ambition,1598.0,90.0,"Skoda Rapid 1.6TDI, 90CP, DSG, Climatronic, Pi...",7990.00,194000.0,Diesel,2015.0
12696,12696,Mercedes-Benz,CLS,Mercedes-Benz CLS 350 d 4MATIC SB Aut,2987.0,258.0,350d 4Matic AMG Restyling 2016 258 CP E6 Distr...,29000.00,120000.0,Diesel,2016.0
12698,12698,BMW,Seria 7,BMW Seria 7 740d xDrive MHEV,2993.0,300.0,BMW 740d xDrive Sedan,119452.41,5000.0,Diesel,2025.0
12699,12699,Volkswagen,Polo,Volkswagen Polo 1.2 Attractive,1198.0,55.0,Volkswagen Polo 1.2 benzină 2007 recent adus d...,1850.00,188000.0,Benzina,2006.0


In [5]:
print(df.columns.tolist())

['Unnamed: 0', 'brand', 'model', 'full_title', 'engine_displacement_cm3', 'power_hp', 'ad_description', 'price_eur', 'mileage_km', 'fuel_type', 'production_year']


In [6]:
df_clean = df.copy()
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

# Example
df_clean = remove_outliers_iqr(df_clean, 'price_eur')
df_clean = remove_outliers_iqr(df_clean, 'mileage_km')
df_clean = remove_outliers_iqr(df_clean, 'engine_displacement_cm3')
df_clean = remove_outliers_iqr(df_clean, 'power_hp')

In [7]:

df_model = df_clean.copy()

df_model = pd.get_dummies(df_model, columns=['brand', 'model', 'fuel_type', 'full_title'], drop_first = True)

'price_eur' in df_model.columns
'ad_description' in df_model.columns

X = df_model.drop(['price_eur', 'ad_description'], axis=1)
y = df_model['price_eur']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [10]:
print("Intercept:", model.intercept_)
print("Number of coefficients:", len(model.coef_))

Intercept: -2216562.997057621
Number of coefficients: 5541


In [11]:
y_pred = model.predict(X_test)

In [12]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2 score:", r2)

MSE: 38013714.0075806
R2 score: 0.8531721081528352


In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Remove outliers
def remove_extreme_outliers(df, column, factor=3):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

def clean_data(df):
    df = df.copy()
    
    # Filter ranges 
    df = df[(df['price_eur'] >= 500) & (df['price_eur'] <= 200000)]
    df = df[(df['mileage_km'] >= 0) & (df['mileage_km'] <= 500000)]
    df = df[(df['engine_displacement_cm3'].isna()) | 
            ((df['engine_displacement_cm3'] >= 500) & (df['engine_displacement_cm3'] <= 6500))]
    df = df[(df['power_hp'].isna()) | ((df['power_hp'] >= 30) & (df['power_hp'] <= 800))]
    
    # Remove extreme outliers using 3x IQR
    for col in ['price_eur', 'mileage_km', 'engine_displacement_cm3', 'power_hp']:
        if col in df.columns:
            df = remove_extreme_outliers(df, col, factor=3)
    
    # Strip whitespace from column names
    df.columns = df.columns.str.strip()
    
    return df

# Reshape fields, like turning production year into a more valuable value in the form of the age of the car
def engineer_features(df):
    df = df.copy()
    current_year = pd.Timestamp.now().year
    
    # Car age
    df['age'] = current_year - df['production_year']
    
    # Mileage per year
    df['km_per_year'] = df['mileage_km'] / df['age'].replace(0,1)
    
    # Power per liter
    df['power_per_liter'] = df['power_hp'] / df['engine_displacement_cm3'].replace(0,1)
    
    return df


# Encode top-N categorical variables
def encode_top_categories(df, col_name, top_n=10, target='price_eur'):
    df = df.copy()
    top_categories = df[col_name].value_counts().head(top_n).index
    df[col_name + '_encoded'] = df[col_name].where(df[col_name].isin(top_categories), 'Other')
    
    # Target encoding
    mean_price = df.groupby(col_name + '_encoded')[target].mean()
    df[col_name + '_encoded'] = df[col_name + '_encoded'].map(mean_price)
    
    return df

def encode_fuel_type(df):
    df = df.copy()
    if 'fuel_type' in df.columns:
        df = pd.get_dummies(df, columns=['fuel_type'], drop_first=True)
    return df

# Prepare X/y
def prepare_xy(df):
    df = df.copy()
    
    # Drop text columns
    drop_cols = ['ad_description', 'full_title', 'brand', 'model']
    for c in drop_cols:
        if c in df.columns:
            df.drop(c, axis=1, inplace=True)
    
    # Fill NaNs with median
    for col in df.select_dtypes(include=['float', 'int']).columns:
        df[col] = df[col].fillna(df[col].median())
    
    # Separate target
    y = np.log(df['price_eur'])  
    X = df.drop('price_eur', axis=1)
    
    return X, y

# Apply all steps
def process_and_split(df, top_brand_n=10, top_model_n=10):
    df = clean_data(df)
    df = engineer_features(df)
    
    # Encode top brands/models
    if 'brand' in df.columns:
        df = encode_top_categories(df, 'brand', top_n=top_brand_n)
    if 'model' in df.columns:
        df = encode_top_categories(df, 'model', top_n=top_model_n)
    
    df = encode_fuel_type(df)

    X, y = prepare_xy(df)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

# Train Linear Regression
def train_and_evaluate(df):
    X_train, X_test, y_train, y_test = process_and_split(df)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(np.exp(y_test), np.exp(y_pred))
    r2 = r2_score(np.exp(y_test), np.exp(y_pred))
    
    print("MSE:", mse)
    print("RMSE:", np.sqrt(mse))
    print("R2 score:", r2)
    
    return model, X_train.columns

model, feature_columns = train_and_evaluate(df)


MSE: 32192196.02013023
RMSE: 5673.8167066032565
R2 score: 0.8739279226358866


In [14]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest(df, n_estimators=200, max_depth=None):
    # Preprocess dataset
    X_train, X_test, y_train, y_test = process_and_split(df)
    
    # Random Forest
    rf = RandomForestRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth, 
                               random_state=42,
                               n_jobs=-1)
    
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    
    # Evaluate
    mse = mean_squared_error(np.exp(y_test), np.exp(y_pred))
    r2 = r2_score(np.exp(y_test), np.exp(y_pred))
    
    print("Random Forest Results:")
    print("MSE:", mse)
    print("RMSE:", np.sqrt(mse))
    print("R2 score:", r2)
    
    return rf, X_train.columns

rf_model, features = train_random_forest(df)

Random Forest Results:
MSE: 20276329.91539945
RMSE: 4502.924595793211
R2 score: 0.9205932073675231
